In [ ]:
from backtest_lib.market.polars_impl import SeriesUniverseMapping

vec1 = SeriesUniverseMapping.from_vectors(["a", "b", "c"], [1, 2, 3])
vec2 = SeriesUniverseMapping.from_vectors(["a", "b", "c"], [1, 2, 3])
vec1 + vec2

In [ ]:
vec3 = SeriesUniverseMapping.from_vectors(["a", "c", "b"], [0, 0, 1])
res = vec1 + vec3
print(res)
assert dict(res) == {"a": 1, "b": 3, "c": 3}

In [ ]:
try:
    vec4 = SeriesUniverseMapping.from_vectors(["a", "c", "d"], [0, 0, 1])
    vec1 + vec4
    assert False
except:
    print("didn't work. that is expected, keys are different.")

In [ ]:
vec5 = SeriesUniverseMapping.from_vectors(["a", "b"], [0, 1])
res = vec1 + vec5
print(res)
assert dict(res) == {"a": 1, "b": 3, "c": 3}


In [ ]:
vec6 = SeriesUniverseMapping.from_vectors(["a", "b"], [2, 2])
res = vec1 * vec6
print(res)

# NOTE: Interesting case here, should "c" be 0 or 3?
assert dict(res) == {"a": 2, "b": 4, "c": 0}

In [ ]:
vec7 = SeriesUniverseMapping.from_vectors(["a", "c"], [2, 2])
res = vec1 / vec7
print(res)

# NOTE: Another sharp edge, should we implicitly pad RHS with 0s?
# It feels weirdly un-ergonomic for division.
assert dict(res) == {"a": 0.5, "b": float("inf"), "c": 1.5}

In [ ]:
from time import perf_counter

acc = SeriesUniverseMapping.from_vectors(["a", "b", "c"], [1, 1, 1])
one = SeriesUniverseMapping.from_vectors(["a", "b", "c"], [1, 1, 1])
one_diff_order = SeriesUniverseMapping.from_vectors(["c", "a", "b"], [1, 1, 1])


# Same order allows for simple vectorised ops
start_same_order = perf_counter()
for i in range(20000):
    acc += one
end_same_order = perf_counter()
print(f"Same order: {end_same_order - start_same_order}s")

# Different key ordering forces us to re-order the keys before performing the vector ops
start_diff_order = perf_counter()
for i in range(20000):
    acc += one_diff_order
end_diff_order = perf_counter()
print(f"Diff order: {end_diff_order - start_diff_order}s")

print(acc)

In [ ]:
keys = [str(i) for i in range(1000)]
diff_keys = reversed(keys)
values = [1] * 1000

print(keys)
long_acc = SeriesUniverseMapping.from_vectors(keys, values)
long_ones = SeriesUniverseMapping.from_vectors(keys, values)
long_ones_diff_order = SeriesUniverseMapping.from_vectors(diff_keys, values)


# Same order allows for simple vectorised ops
start_same_order = perf_counter()
for i in range(1000):
    long_acc += long_ones
end_same_order = perf_counter()
print(f"Same order: {end_same_order - start_same_order}s")

# Worst case: With a longer set of keys, the re-ordering takes a VERY long time
start_diff_order = perf_counter()
for i in range(1000):
    long_acc += long_ones_diff_order
end_diff_order = perf_counter()
print(f"Diff order: {end_diff_order - start_diff_order}s")

print(long_acc)